<a href="https://colab.research.google.com/github/skywalker00001/Conterfactual-Reasoning-Project/blob/main/time_travel2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = 'drive/MyDrive/LM/'

Mounted at /content/drive


In [2]:
!pip install simplet5

     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 2.8 MB 63.4 MB/s 
     |████████████████████████████████| 919 kB 72.3 MB/s 
     |████████████████████████████████| 596 kB 78.4 MB/s 
     |████████████████████████████████| 396 kB 74.4 MB/s 
     |████████████████████████████████| 829 kB 75.8 MB/s 
     |████████████████████████████████| 133 kB 72.6 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 67.0 MB/s 
     |████████████████████████████████| 3.3 MB 52.1 MB/s 
     |████████████████████████████████| 1.1 MB 64.6 MB/s 
     |████████████████████████████████| 271 kB 69.8 MB/s 
     |████████████████████████████████| 192 kB 66.6 MB/s 
     |████████████████████████████████| 160 kB 77.5 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.3-py3-none-any.whl size=6832 sha256=a4d8e5a60af97d4cce1d23d1c13f4075f5ee9c19a4bf8ea003b181840ebac7c5
  Stored in directory: /root/.cache/pi

In [3]:
import transformers
transformers.models.auto.__spec__  = True
from simplet5 import SimpleT5
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import regex as re

Global seed set to 42


In [4]:
small_path = root + '/TimeTravel/train_supervised_small.json'
small_df = pd.read_json(small_path, lines=True)
small_df.head()

,story_id,premise,initial,counterfactual,original_ending,edited_ending
0,080198fc-d0e7-42b3-8e63-b2144e59d816,On my way to work I stopped to get some coffee.,I went through the drive through and placed my...,I went inside to place my order.,I paid the cashier and patiently waited for my...,[I paid the cashier and patiently waited at th...
1,1ba02a18-8807-4f39-9271-ef555597ce21,Terry aspired to be a chef.,His father is one.,He moved to Italy and opened a restaurant.,He decided he would continue the business. He ...,"[He decided he would continue the business., H..."
2,dbb0ad3e-9389-44ee-8290-7c3458e3fa0f,Kim and her glass went on a field trip to an a...,Everyone enjoyed looking at the sea creatures.,Everyone did not enjoy looking at the sea cre...,"But when they went to the shark exhibit, Kim w...",[Instead of going home they went to the shark ...
3,e5955040-5b87-4acb-a8c7-7e81d0ffb9f5,Susie was sitting on her barstool.,She kept kicking the counter with her feet.,She kept herself steady with her feet.,"Suddenly, her kick sent her falling backwards....","[Suddenly, an earthquake sent her falling back..."
4,dc234072-2e69-4999-9e2f-632d3ea30b78,Celeste rode her motorcycle across the woods.,"When she almost arrived at the intersection, a...","When she almost arrived at the intersection, a...",She didn't have enough time to brake and drove...,[She didn't have enough time to brake and drov...


In [5]:
print(small_df.loc[:,"edited_ending"][0])

['I paid the cashier and patiently waited at the counter for my drink.', 'When she handed me the drink, the lid came off and spilled on me.', 'The coffee hurt and I had to go home and change clothes.']


In [9]:
text_a, text_b = [], []

for i in range(len(small_df)):
  text_a.append("premise: " + small_df.loc[i, 'premise'] + " initial: " + \
                small_df.loc[i, 'initial'] + " counterfactual: " + small_df.loc[i, 'counterfactual'] + \
                " original_ending: " + small_df.loc[i, 'original_ending'])
  #text_a.append(re.sub(re_pat, df.loc[i, 'edit1'], df.loc[i, 'original1']))
  text_b.append("edited_ending: " + small_df.loc[i, 'edited_ending'][0] +" "+ small_df.loc[i, 'edited_ending'][1] +""+ \
                small_df.loc[i, 'edited_ending'][2])

In [10]:
processed_df = pd.DataFrame({'source_text': text_a, 'target_text': text_b}) 
processed_df.head()

,source_text,target_text
0,premise: On my way to work I stopped to get so...,edited_ending: I paid the cashier and patientl...
1,premise: Terry aspired to be a chef. initial: ...,edited_ending: He decided he would continue th...
2,premise: Kim and her glass went on a field tri...,edited_ending: Instead of going home they went...
3,premise: Susie was sitting on her barstool. in...,"edited_ending: Suddenly, an earthquake sent he..."
4,premise: Celeste rode her motorcycle across th...,edited_ending: She didn't have enough time to ...


In [11]:
print(processed_df.loc[0, "source_text"])
print("-------------")
print(processed_df.loc[0, "target_text"])

premise: On my way to work I stopped to get some coffee. initial: I went through the drive through and placed my order. counterfactual: I went inside to place my order. original_ending: I paid the cashier and patiently waited for my drink. When she handed me the drink, the lid came off and spilled on me. The coffee hurt and I had to go home and change clothes.
-------------
edited_ending: I paid the cashier and patiently waited at the counter for my drink. When she handed me the drink, the lid came off and spilled on me.The coffee hurt and I had to go home and change clothes.


In [13]:
print(processed_df.shape)

(16752, 2)


In [14]:
train_df, test_df = train_test_split(processed_df, test_size=0.2)
train_df.shape, test_df.shape

((13401, 2), (3351, 2))

# Train using T5

In [ ]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=128, 
            target_max_token_len=128, 
            batch_size=16, max_epochs=80, use_gpu=True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

In [ ]:
!ls ./outputs

simplet5-epoch-0-train-loss-0.9084   simplet5-epoch-23-train-loss-0.1428
simplet5-epoch-10-train-loss-0.3904  simplet5-epoch-24-train-loss-0.1321
simplet5-epoch-11-train-loss-0.3597  simplet5-epoch-25-train-loss-0.122
simplet5-epoch-12-train-loss-0.3337  simplet5-epoch-26-train-loss-0.1142
simplet5-epoch-13-train-loss-0.3089  simplet5-epoch-27-train-loss-0.1052
simplet5-epoch-14-train-loss-0.2855  simplet5-epoch-28-train-loss-0.0976
simplet5-epoch-15-train-loss-0.264   simplet5-epoch-29-train-loss-0.0917
simplet5-epoch-16-train-loss-0.2445  simplet5-epoch-2-train-loss-0.734
simplet5-epoch-17-train-loss-0.2279  simplet5-epoch-3-train-loss-0.6776
simplet5-epoch-18-train-loss-0.2091  simplet5-epoch-4-train-loss-0.6281
simplet5-epoch-19-train-loss-0.193   simplet5-epoch-5-train-loss-0.5771
simplet5-epoch-1-train-loss-0.7947   simplet5-epoch-6-train-loss-0.5316
simplet5-epoch-20-train-loss-0.1804  simplet5-epoch-7-train-loss-0.4921
simplet5-epoch-21-train-loss-0.1649  simplet5-epoch-8-train

In [ ]:

models_dir = "./outputs/"
ckpt_files = os.listdir(models_dir)
print('All files are:')
print(f'\t{ckpt_files}')
print(len(ckpt_files))

loss_epoch = re.compile(r'loss-\d+\.\d+') # 正则表达式，匹配-val后面的数字，代表第几个epoch
ckpt_files_sorted = sorted(ckpt_files, key=lambda t: re.findall(loss_epoch, t)[0], reverse=False) # 按照valxxx的xxx进行倒叙排序，因为最新的epoch才保存最好的model
print('Sorting .ckpt files:')
print(f'\t{ckpt_files_sorted}')
print()

print('Best model file is:')
best_model_name = ckpt_files_sorted[0]
print(f'\t{best_model_name}')

All files are:
	['simplet5-epoch-9-train-loss-0.4207', 'simplet5-epoch-1-train-loss-0.7947', 'simplet5-epoch-4-train-loss-0.6281', 'simplet5-epoch-27-train-loss-0.1052', 'simplet5-epoch-7-train-loss-0.4921', 'simplet5-epoch-2-train-loss-0.734', 'simplet5-epoch-6-train-loss-0.5316', 'simplet5-epoch-18-train-loss-0.2091', 'simplet5-epoch-13-train-loss-0.3089', 'simplet5-epoch-11-train-loss-0.3597', 'simplet5-epoch-17-train-loss-0.2279', 'simplet5-epoch-10-train-loss-0.3904', 'simplet5-epoch-20-train-loss-0.1804', 'simplet5-epoch-0-train-loss-0.9084', 'simplet5-epoch-14-train-loss-0.2855', 'simplet5-epoch-29-train-loss-0.0917', 'simplet5-epoch-22-train-loss-0.1546', 'simplet5-epoch-8-train-loss-0.4563', 'simplet5-epoch-5-train-loss-0.5771', 'simplet5-epoch-26-train-loss-0.1142', 'simplet5-epoch-28-train-loss-0.0976', 'simplet5-epoch-21-train-loss-0.1649', 'simplet5-epoch-24-train-loss-0.1321', 'simplet5-epoch-3-train-loss-0.6776', 'simplet5-epoch-12-train-loss-0.3337', 'simplet5-epoch-16-

In [ ]:
best_model_name

'simplet5-epoch-29-train-loss-0.0917'

In [ ]:
# let's load the trained model for inferencing:
model.load_model("t5","outputs/" + best_model_name, use_gpu=True)

# Save the model

In [ ]:
import shutil 

def copy_allfiles(src,dest):
#src:原文件夹；dest:目标文件夹
  src_files = os.listdir(src)
  for file_name in src_files:
    full_file_name = os.path.join(src, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, dest)


In [ ]:
model_iter = "model2.1"
copy_allfiles("outputs/" + best_model_name, root + "models/" + model_iter)

In [ ]:
# model.load_model("t5",root+'models/model2.1', use_gpu=True)

# Inference

In [ ]:
large_path = root + '/TimeTravel/train_supervised_large.json'
df_large = pd.read_json(large_path, lines=True)
print(len(df_large))

28363


In [ ]:
small_ids = []
for i in range(len(df)):
    small_ids.append(df.loc[i, 'story_id'])

print(len(small_ids))

16752


In [ ]:
df_large = df_large[~df_large.story_id.isin(small_ids)]
df_large = df_large.reset_index()   # must reset index after delete rows
print(len(df_large))

11613


In [ ]:
# select data not in training set
part_df_large = df_large[0:1000]
df_large = df_large.reset_index()
print(len(part_df_large))

1000


In [ ]:
part_df_large.head(10)

,index,story_id,premise,initial,counterfactual,original_ending,edited_ending
0,4183,aec50dac-80ba-4a0b-9ef5-91a10da0191b,Julia was known as a mean girl.,She treated all her friends horribly.,But she is very kind to her friends.,"Soon, none of them wanted to spend time with h...","[Soon,everyone wanted to spend time with her.,..."
1,4184,97466060-d806-4bdd-8bdd-9bb7b4b6a75f,The opposing team scored a three pointer at th...,Their team won the championship.,Their team still lost the championship though.,Bill congratulated the players. He also went i...,"[Bill congratulated the other team., He also w..."
2,4185,21d2bd4d-7927-421e-b3e0-d5e97ce97b54,Cara and her brother were playing baseball.,But she hit the ball too hard!,"She got angry, and stormed off with the ball.",It flew over the fence and vanished. Without a...,"[She jumped over the fence and vanished., With..."
3,4186,ea6530d9-46f9-4d43-8404-039979be8a2c,Janet always wanted to be an EMT.,She was excited to start working with a team o...,She was scared to start working with a team of...,She enjoyed helping the people and making them...,[She was nervous about helping the people and ...
4,4187,5c630fc4-b907-4732-be42-a64de9d9ee1b,There wasn't much work to do this morning.,We almost got into an argument because of it.,I didnt come in until noon.,I tried to stay quiet and not start a problem....,"[I tried to stay quiet and busy., I knew I cou..."
5,4188,02514eea-0dbf-46f9-af9f-4dc02577a212,Pam is married to her Husband Jeff.,Things have been really rocky as of late.,Things have been going really great lately.,Jeff isn't the same person anymore. Pam decide...,[Jeff then became distant and wasn't the same ...
6,4189,97b3597f-e1c2-4599-87c9-3e1792584f9d,I went out to buy a new suit yesterday.,The salesman asked me all sorts of questions.,The salesman recommended I buy a nice green suit.,Then the salesman took my measurements. After ...,"[Then the salesman took my measurements., Afte..."
7,4190,3fdc4fda-d9b4-4355-b0ff-5196aadb4dd3,Michael was driving to a concert.,His car begin to make a loud noise.,The noise was coming from the exhaust pipe.,He decides to pull over and see what's wrong. ...,[He decides to pull over and see what's wrong....
8,4191,d037216c-837c-4cdb-866d-72460979d020,The band marched for peace.,One band member wasn't looking where he was go...,One band member dropped his instrument.,He ran into a pedestrian. The pedestrian thoug...,"[It landed on a pedestrian's foot., The pedest..."
9,4192,26234c62-c70d-4953-b04d-d20cdde98f29,Kelley felt sick.,She wanted to go to the party but was not up t...,She decided to stay home and slept all night.,"Yet, because she felt obliged, she went anyway...","[She felt obliged, but she chose not to go., A..."


In [ ]:
text, gt = [],[] # gt for ground truth

for i in range(len(part_df_large)):
  text.append("premise: " + part_df_large.loc[i, 'premise'] + \
              " initial: " + part_df_large.loc[i, 'initial'] + \
              " counterfactual: " + part_df_large.loc[i, 'counterfactual'] + \
              " original_ending: " + part_df_large.loc[i, 'original_ending'])
  gt.append("edited_ending: " + part_df_large.loc[i, 'edited_ending'][0] +" "+ \
            part_df_large.loc[i, 'edited_ending'][1] +""+ part_df_large.loc[i, 'edited_ending'][2])

print(len(text))

In [ ]:
%time
#origin = []
edit_endings = []
for i in tqdm(range(len(text))):
    #origin.append(processed_un_df.loc[i, 'raw_text'])
    edit_endings.extend(model.predict(text[i]))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


100%|██████████| 1000/1000 [09:26<00:00,  1.76it/s]


In [ ]:
out_df = pd.DataFrame({'raw_text': text, 'ground_truth': gt, 'generated_text': edit_endings}) 

In [ ]:
out_df.to_csv(root+'output2.1.csv')
out_df.to_json(root+'output2.1.json')

In [ ]:
import difflib

#判断相似度的方法，用到了difflib库
def get_equal_rate_1(str1, str2):
   return difflib.SequenceMatcher(None, str1, str2).quick_ratio()

# Select some samples

In [ ]:
print(df[df.story_id == "0930f02c-df49-480c-be22-91db02976e01"])

                                  story_id  ...                                      edited_ending
1336  0930f02c-df49-480c-be22-91db02976e01  ...  [I was so happy to see her., I breathed a sigh...

[1 rows x 6 columns]


In [ ]:
samp = df[df.story_id == "0930f02c-df49-480c-be22-91db02976e01"]
sample1 = "premise: " + samp.premise.item() + \
              " initial: " + samp.initial.item() + \
              " counterfactual: " + samp.counterfactual.item() + \
              " original_ending: " + samp.original_ending.item()
print(sample1)

In [ ]:
model.predict(sample1)

["edited_ending: I was so freaked out, I ran to my kitchen and opened the freezer. I breathed a sigh of absolute relief.She was still there, where I'd left her."]